In [1]:
# dataset
import pandas as pd
features = pd.read_excel('data.xlsx')
features.head()

,CVE_ID,exploit_date,label,zeroday_price,date_entry_created,original_release_date,overview,CVSS_V3_base_score,CVSS_V3_impact_score,CVSS_V3_exploitability_score,...,CVSS_V2_impact_type,NVD_reference_count,NVD_vulnerable_software_and_versions_count,NVD_CWEID,cvedetail_affected total number(all products and versions),cvedetail_affected vendor,cvedetail_affected vendor number,cvedetail_affected product,cvedetail_affected product number,cvedetail_vultype
0,CVE-2000-1254,NaT,0,$10k-$25k,2016-05-04,2016-05-04,crypto/rsa/rsa_gen.c in OpenSSL before 0.9.6 m...,7.5,3.6,3.9,...,Allows unauthorized disclosure of information...,5,1,CWE-310,1.0,Openssl,1.0,Openssl,1.0,NaN
1,CVE-2005-4900,NaT,0,NaN,2016-10-14,2016-10-14,"SHA-1 is not collision resistant, which makes ...",5.9,3.6,2.2,...,Allows unauthorized disclosure of information...,5,1,CWE-326,1.0,Google,1.0,Chrome,1.0,NaN
2,CVE-2010-5325,NaT,0,$2k-$5k,2016-02-15,2016-04-15,Heap-based buffer overflow in the unhtmlify fu...,9.8,5.9,3.9,...,Allows unauthorized disclosure of information...,7,4,CWE-119,8.0,"Linuxfoundation, Oracle, Redhat",3.0,"Foomatic-filters, Linux, Enterprise Linux, Ent...",8.0,Denial Of Service
3,CVE-2010-5326,NaT,0,$10k-$25k,2016-05-12,2016-05-13,The Invoker Servlet on SAP NetWeaver Applicati...,10.0,6.0,3.9,...,Allows unauthorized disclosure of information...,6,1,NVD-CWE-noinfo,1.0,SAP,1.0,Netweaver,1.0,Execute Code
4,CVE-2011-5326,NaT,0,$0k-$1k,2016-04-10,2016-05-13,imlib2 before 1.4.9 allows remote attackers to...,7.5,3.6,3.9,...,Allows disruption of service,5,2,CWE-189,3.0,"Debian, Enlightenment",2.0,"Debian Linux, Imlib2",2.0,Denial Of Service


# Calculate the count of CVE disclose time each month

In [2]:
from datetime import datetime
count_cve_disclose=[0,0,0,0,0,0,0]
month=['5','6','7','8','9','10','11']
for index, row in features.iterrows():
    if index<6000:
        year=str(row['date_entry_created'])
        
        if year.find('2016')>-1:
            try:
                time=str(row['date_entry_created'])
                time=datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
                m=str(time.month)
                for k in range(len(month)):
                    if month[k]==m:
                        count_cve_disclose[k]+=1
            except:
                time=''
count_cve_disclose

[466, 472, 201, 288, 238, 96, 64]

# We get our collection of Twitter discussion, and calculate the discussion each month.

In [3]:
month=[5, 6, 7, 8, 9, 10, 11]
count_discuss_on_twitter=[15402, 14206, 20838, 6337, 48387, 122150, 39685]

In [4]:
month

[5, 6, 7, 8, 9, 10, 11]

# draw CVE disclosure plot

In [5]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label

source1 = ColumnDataSource(data=dict(height=count_cve_disclose,
                                    weight=month,
                                    names=count_cve_disclose))

p = figure(plot_width=650, plot_height=400, y_range=(10, 600),title='CVE disclosure each month')

p.line(month, count_cve_disclose, line_width=2,color="red")
p.circle(month, count_cve_disclose, color="red",fill_color="red", size=10)
p.xaxis[0].axis_label = 'month'
p.yaxis[0].axis_label = 'CVE disclosure count'

labels = LabelSet(x='weight', y='height', text='height', level='glyph',
              x_offset=-5, y_offset=10, source=source1, render_mode='canvas',text_font_size='6pt')

p.left[0].formatter.use_scientific = False
p.add_layout(labels)
output_notebook()
show(p)

Loading BokehJS ...

# draw the discussion on Twitter plot

In [6]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label

source2 = ColumnDataSource(data=dict(height=count_discuss_on_twitter,
                                    weight=month,
                                    names=count_discuss_on_twitter))

p = figure(plot_width=720, plot_height=400, y_range=(100, 150000),title='CVE discussion each month on Twitter')

p.line(month, count_discuss_on_twitter, line_width=2,color="blue")
p.circle(month, count_discuss_on_twitter, color="blue",fill_color="blue", size=10)
p.xaxis[0].axis_label = 'month'
p.yaxis[0].axis_label = 'CVE discussion count on Twitter'

labels = LabelSet(x='weight', y='height', text='height', level='glyph',
              x_offset=-5, y_offset=10, source=source2, render_mode='canvas',text_font_size='6pt')

p.left[0].formatter.use_scientific = False
p.add_layout(labels)
output_notebook()
show(p)

Loading BokehJS ...

# Combine two plot

In [7]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label, LinearAxis

source2 = ColumnDataSource(data=dict(height=count_discuss_on_twitter,
                                    weight=month,
                                    names=count_discuss_on_twitter))

p = figure(plot_width=700, plot_height=500, y_range=(100, 150000),title='CVE discussion each month on Twitter')
p.extra_y_ranges = {"foo": Range1d(start=0, end=500)}
p.add_layout(LinearAxis(y_range_name="foo"), 'right')

p.line(month, count_cve_disclose, line_width=2,color="red", y_range_name="foo")
p.circle(month, count_cve_disclose, color="red",fill_color="red", size=10, y_range_name="foo")

p.line(month, count_discuss_on_twitter, line_width=2,color="blue")
p.circle(month, count_discuss_on_twitter, color="blue",fill_color="blue", size=10)

p.xaxis[0].axis_label = 'month'
p.yaxis[0].axis_label = 'CVE discussion count on Twitter'

# add label on pot
labels1 = LabelSet(x='weight', y='height', text='height', level='glyph',
              x_offset=-5, y_offset=10, source=source1, render_mode='canvas',text_font_size='6pt', y_range_name="foo")
labels2 = LabelSet(x='weight', y='height', text='height', level='glyph',
              x_offset=-5, y_offset=10, source=source2, render_mode='canvas',text_font_size='6pt')

p.add_layout(labels1)
p.add_layout(labels2)

# Scientific mark
p.left[0].formatter.use_scientific = False

output_notebook()
show(p)

Loading BokehJS ...